In [1]:
import pandas as pd 
import os 
import seaborn as sns
import matplotlib.pyplot as plt

# Set the style of the plots
markers = ['o', 's', 'D', 'v', 'p', 'P', 'X', 'h', 'H', 'd']
colors_discrete = sns.color_palette('tab10')
colors_continuous = sns.color_palette('viridis')


# Do not hide any columns
pd.set_option('display.max_columns', None)

results_path = 'results_combined/results_wine1/results'

# Read all the results from results/ directory
results = []
for file in os.listdir(results_path):
    if file.endswith('.feather'):
        results.append(pd.read_feather(f'{results_path}/{file}'))

# Concatenate all the results
df: pd.DataFrame = pd.concat(results, ignore_index=True)
print(df.info())
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32400 entries, 0 to 32399
Data columns (total 47 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   experiment_type                                 32400 non-null  object 
 1   dataset_name                                    32400 non-null  object 
 2   k_mlps_in_B                                     32400 non-null  int64  
 3   fold_i                                          32400 non-null  int64  
 4   experiment_generalization_type                  32400 non-null  object 
 5   beta_confidence                                 32400 non-null  float64
 6   delta_robustness                                32400 non-null  float64
 7   model2_name                                     32400 non-null  object 
 8   x_test_sample                                   32400 non-null  object 
 9   y_test_sample                          

,experiment_type,dataset_name,k_mlps_in_B,fold_i,experiment_generalization_type,beta_confidence,delta_robustness,model2_name,x_test_sample,y_test_sample,model1_pred_proba,model1_pred_crisp,model2_pred_proba,model2_pred_crisp,base_counterfactual,base_counterfactual_model1_pred_proba,base_counterfactual_model1_pred_crisp,base_counterfactual_model2_pred_proba,base_counterfactual_model2_pred_crisp,base_counterfactual_validity,base_counterfactual_proximityL1,base_counterfactual_proximityL2,base_counterfactual_plausibility,base_counterfactual_discriminative_power,base_counterfactual_validity_model2,base_counterfactual_time,robust_counterfactual,robust_counterfactual_model1_pred_proba,robust_counterfactual_model1_pred_crisp,robust_counterfactual_model2_pred_proba,robust_counterfactual_model2_pred_crisp,robust_counterfactual_validity,robust_counterfactual_proximityL1,robust_counterfactual_proximityL2,robust_counterfactual_plausibility,robust_counterfactual_discriminative_power,robust_counterfactual_validity_model2,robust_counterfactual_L1_distance_from_base_cf,robust_counterfactual_L2_distance_from_base_cf,robust_counterfactual_time,start_sample_passes_test,counterfactual_does_not_pass_test,counterfactual_does_not_have_target_class,counterfactual_is_nan,highest_delta,lower_bound_beta,upper_bound_beta
0,Seed-Architecture,wine_quality,32,2,Seed-Architecture,0.95,0.8,Model2_11,"[0.5423728813559321, 0.22666666666666663, 0.34...",0,0.710993,1,0.463754,0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.5311615470587239, 0.2203955595535264, 0.339...",0.473287,0.0,0.417556,0.0,1.0,0.13262,0.047589,0.167785,0.333333,1.0,0.0,0.0,0.029860,1.0,0.0,0.0,0.0,0.925072,NaN,NaN
1,Seed-Architecture,wine_quality,32,2,Seed-Architecture,0.95,0.8,Model2_12,"[0.5423728813559321, 0.22666666666666663, 0.34...",0,0.710993,1,0.545659,1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.5311615470587239, 0.2203955595535264, 0.339...",0.473287,0.0,0.395465,0.0,1.0,0.13262,0.047589,0.167785,0.333333,1.0,0.0,0.0,0.029054,1.0,0.0,0.0,0.0,0.925072,NaN,NaN
2,Seed-Architecture,wine_quality,32,2,Seed-Architecture,0.95,0.8,Model2_13,"[0.5423728813559321, 0.22666666666666663, 0.34...",0,0.710993,1,0.497242,0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.5311615470587239, 0.2203955595535264, 0.339...",0.473287,0.0,0.455864,0.0,1.0,0.13262,0.047589,0.167785,0.333333,1.0,0.0,0.0,0.029201,1.0,0.0,0.0,0.0,0.925072,NaN,NaN
3,Seed-Architecture,wine_quality,32,2,Seed-Architecture,0.95,0.8,Model2_14,"[0.5423728813559321, 0.22666666666666663, 0.34...",0,0.710993,1,0.468102,0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.5311615470587239, 0.2203955595535264, 0.339...",0.473287,0.0,0.428203,0.0,1.0,0.13262,0.047589,0.167785,0.333333,1.0,0.0,0.0,0.028548,1.0,0.0,0.0,0.0,0.925072,NaN,NaN
4,Seed-Architecture,wine_quality,32,2,Seed-Architecture,0.95,0.8,Model2_15,"[0.5423728813559321, 0.22666666666666663, 0.34...",0,0.710993,1,0.494516,0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.5311615470587239, 0.2203955595535264, 0.339...",0.473287,0.0,0.358790,0.0,1.0,0.13262,0.047589,0.167785,0.333333,1.0,0.0,0.0,0.028537,1.0,0.0,0.0,0.0,0.925072,NaN,NaN


In [2]:
df['start_sample_passes_test'].value_counts()

0.0    15990
1.0     5610
Name: start_sample_passes_test, dtype: int64

In [6]:
base_validity_model2 = df[df['base_counterfactual'] != None]['base_counterfactual_validity_model2']
print('Average base_counterfactual_validity_model2:', base_validity_model2.mean())
print('Notna %:', base_validity_model2.notna().sum() / base_validity_model2.shape[0])

robust_validity_model2 = df[df['robust_counterfactual'] != None]['robust_counterfactual_validity_model2']
print('Average robust_counterfactual_validity_model2:', robust_validity_model2.mean())
print('Notna %:', robust_validity_model2.notna().sum() / robust_validity_model2.shape[0])

Average base_counterfactual_validity_model2: 0.44
Notna %: 0.16666666666666666
Average robust_counterfactual_validity_model2: 0.9031018518518519
Notna %: 0.6666666666666666


In [13]:
df[df['delta_robustness'] == 0.9]['base_counterfactual_model1_pred_proba'].value_counts()

Series([], Name: base_counterfactual_model1_pred_proba, dtype: int64)

0    True
1    True
2    True
3    True
4    True
Name: base_counterfactual, dtype: bool

In [12]:
import numpy as np

robust = []
robust_ci = []
base = []
base_ci = []

unique_deltas = sorted(df['delta_robustness'].unique())
print(unique_deltas)
confidence = 0.95
z = 1.96  # z-score for 95% confidence

exp1_df = df[df['experiment_type'] == 'Seed-Bootstrap-Architecture']

for delta in unique_deltas:
    robust_data = exp1_df[(exp1_df['delta_robustness'] == delta) & (exp1_df['beta_confidence'] == confidence) & (exp1_df['start_sample_passes_test'] == 1)]['robust_counterfactual_validity_model2'].dropna()
    robust_mean = robust_data.mean()
    robust.append(robust_mean)
    robust_ci.append(z * np.sqrt((robust_mean*(1-robust_mean))/len(robust_data)))

    base_data = exp1_df[(exp1_df['delta_robustness'] == delta) & (exp1_df['beta_confidence'] == confidence)]['base_counterfactual_validity_model2'].dropna()
    base_mean = base_data.mean()
    base.append(base_mean if not pd.isna(base_mean) else base[0])
    base_ci.append(z * np.sqrt((base_mean*(1-base_mean))/len(base_data)) if not pd.isna(base_mean) else 0)

fig, ax = plt.subplots(1, 1, figsize=(8, 5))

plt.title(f'Empirical validity for model 2 @ {int(confidence*100)}% confidence')
plt.xlabel('Delta')
plt.ylabel('Validity for model 2')

plt.errorbar(unique_deltas, robust, yerr=robust_ci, label='Robust counterfactuals', marker=markers[0], color=colors_discrete[0])
plt.errorbar(unique_deltas, base, yerr=base_ci, label='Base counterfactuals', marker=markers[1], color=colors_discrete[1])

plt.legend()
plt.tight_layout()

plt.show()

[0.5, 0.6, 0.7, 0.8, 0.9]


ZeroDivisionError: float division by zero